In [1]:
from rag_assistant.rag_assistant import RAGAssistant
from langchain_classic.retrievers import ParentDocumentRetriever
from langchain_text_splitters import RecursiveCharacterTextSplitter, MarkdownHeaderTextSplitter
import re
from langchain_classic.storage import InMemoryStore

In [19]:
from langchain_classic.storage import LocalFileStore
from langchain_classic.storage._lc_store import create_kv_docstore

# Use a real folder on your computer
fs = LocalFileStore("./parent_docs_folder")
store = create_kv_docstore(fs)

In [2]:
assistant = RAGAssistant(persist_path="./chroma/rag", collection_name="blueprint_text_analytics", topic="Blueprints for Text Analytics in Python textbook")

Device being used in the embedding model is cpu

RAG Assistant initialized successfully for topic: Blueprints for Text Analytics in Python textbook


In [3]:
vector_db = assistant.vector_db

In [9]:
results = vector_db.similarity_search_with_score(query="Summarize chapter 1 for me")

In [14]:
results = vector_db.search(query="Summarize chapter 1 for me", search_type='mmr', k=5)

In [17]:
new_results = assistant._format_docs_with_metadata(results)

In [4]:
with open("data/structured_text.md", "r", encoding='utf-8') as f:
    structured_text = f.read()

In [6]:
markdown_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=[("#", "Chapter"), ("##", "Topic"), ("###", "Blueprint")],
    strip_headers=False
    )

markdown_content = markdown_splitter.split_text(structured_text)

child_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 1200,
        chunk_overlap = 150,
        #add_start_index=True
    )

split_content = child_splitter.split_documents(markdown_content)

# parent_splitter = RecursiveCharacterTextSplitter(
#     chunk_size = 2000,
#     add_start_index=True
# )

# store = InMemoryStore()

# retriever = ParentDocumentRetriever(
#     vectorstore=vector_db,
#     docstore = store,
#     child_splitter=child_splitter,
#     parent_splitter=parent_splitter,
#     search_kwargs={"k": 5, "fetch_k": 25, "lambda_mult": 0.4},
#     search_type='mmr'
# )

In [6]:
retriever.add_documents(markdown_content, ids=None)

In [13]:
my_results = retriever.invoke("Tell me what you know about inverse document frequency")

In [59]:
len(vector_db.get()['documents'])

6984

In [61]:
len(list(store.yield_keys()))

691